In [ ]:
# import sys
# !{sys.executable} -m pip install numpy pandas matplotlib sklearn seaborn
# !{sys.executable} -m pip install --upgrade gensim
# !{sys.executable} -m pip install nltk
# import nltk
# nltk.download('punkt')
# !{sys.executable} -m pip install beautifulsoup4

In [1]:
import os
import re
import gensim
import multiprocessing
import random
import logging
import numpy as np
import zipfile
import tensorflow as tf

from urllib import request

from pathlib import Path
from os import listdir
from os.path import isfile, join
from nltk.tokenize import RegexpTokenizer
from bs4 import BeautifulSoup

from sklearn.cluster import KMeans
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [12]:
SKIP_FILES = ""
NEWLINE = '\n'

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r",", " ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

"""
read training files
"""
def read_files(path):
    print("path: {}...".format(path))
    for root, dirnames, filenames in os.walk(path):
        for dirname in dirnames:
            read_files(os.path.join(root, dirname))
        for filename in filenames:
            if filename not in SKIP_FILES:
                filepath = os.path.join(root, filename)
                if os.path.isfile(filepath):
                    lines = []
                    f = open(filepath, encoding='latin-1')
                    for line in f:
                        lines.append(line)
                    f.close()
                    content = NEWLINE.join(lines)
                    yield filename, content

def download(url, filename):
    """Download a file if not present"""
    if not os.path.exists(filename):
        print("downloading {}...".format(filename))
        filename, _ = request.urlretrieve(url + filename, filename)

        print("extracting {}...".format(filename))
        with zipfile.ZipFile(filename) as f:
            f.extractall()

    return filename

In [13]:
def get_data(url, filename):
    filename = download(url, filename)
    data_path = filename.replace('.zip','')
    documents = []
    print("building documents...")
    for fname, text in read_files(data_path):
        documents.append(clean_str(text).split(' '))
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(documents)]
    print("building documents done")
    return documents

In [14]:
# download data training
documents = get_data(url='https://github.com/kadriansyah/notebook/raw/master/alodokter-doc2vec-article/', filename="data.zip")
print("we have {} documents".format(len(documents)))

downloading data.zip...
extracting data.zip...
building documents...
path: data...
building documents done
we have 7932 documents


In [15]:
documents[0]

TaggedDocument(words=['penyebab', 'bayi', 'muntah', 'setelah', 'minum', 'asi', 'dan', 'cara', 'mengatasinya', 'bayi', 'muntah', 'setelah', 'minum', 'asi', '(', 'air', 'susu', 'ibu', ')', 'adalah', 'keluhan', 'yang', 'sering', 'terjadi', 'sebagian', 'bayi', 'bahkan', 'mengalaminya', 'hampir', 'setiap', 'kali', 'selesai', 'menyusu', 'meski', 'umumnya', 'normal', 'kondisi', 'ini', 'bisa', 'juga', 'disebabkan', 'oleh', 'gangguan', 'berbahaya', 'yang', 'harus', 'diwaspadai', 'bayi', 'muntah', 'setelah', 'minum', 'asi', 'dikenal', 'dengan', 'istilah', 'gumoh', 'gumoh', 'dikatakan', 'normal', 'apabila', 'tidak', 'menyebabkan', 'bayi', 'rewel', 'atau', 'sesak', 'napas', 'meskipun', 'dapat', 'dicegah', 'kondisi', 'tersebut', 'tidak', 'memerlukan', 'penanganan', 'khusus', 'dan', 'normal', 'terjadi', 'penyebab', 'bayi', 'muntah', 'setelah', 'minum', 'asi', 'gumoh', 'disebabkan', 'oleh', 'asi', 'atau', 'susu', 'yang', 'ditelan', 'bayi', 'kembali', 'ke', 'kerongkongan', 'karena', 'otot', 'di', 'sal

In [16]:
def evaluate(model, documents, steps):
    percentiles = np.zeros(steps)
    for step in range(steps):
        docid = np.random.randint(model.docvecs.count)
        inferred_vector = model.infer_vector(documents[docid][0])
        similars = model.docvecs.most_similar(positive=[inferred_vector], topn=10)
        for idx,simdoc in enumerate(similars):
            if simdoc[0] == docid:
                print("found similar document with id {} in position {} with similarity score {}".format(simdoc[0], idx, simdoc[1]))
                percentiles[step] = ((len(similars) - idx) / len(similars)) * 100
                break
    return np.mean(percentiles)

def train(documents=documents, model_name="model/alodokter-articles-doc2vec.model", max_epochs=50, patience=3):
    best_mean_percentiles = 0
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
    model = Doc2Vec(dm=1, vector_size=300, window=2, alpha=0.1, min_alpha=0.0001, min_count=5, epochs=1, workers=5)
    model.build_vocab(documents)
    for epoch in range(max_epochs):
        print('training epoch {:d} ...'.format(epoch))
        model.train(documents, total_examples=model.corpus_count,epochs=model.epochs)
        mean_percentiles = evaluate(model,documents,10)
        print('mean percentiles: {:.2f}'.format(mean_percentiles))
        
        if mean_percentiles < best_mean_percentiles:
            print("current mean_percentiles: {:.2f}, best: {:.2f}".format(mean_percentiles, best_mean_percentiles))
            patience = patience-1
        else:
            best_mean_percentiles = mean_percentiles
            print("========== Saving best model with mean_percentiles: {:.2f} ==========".format(mean_percentiles))
            model.save(model_name)
            patience = patience+1
        
        if patience == 0:
            print("early stop...")
            print("========== Saving best model with mean_percentiles: {:.2f} ==========".format(best_mean_percentiles))
            break
    
    return model

In [17]:
model = train(documents)

2020-02-23 08:41:29,648 : INFO : collecting all words and their counts
2020-02-23 08:41:29,649 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2020-02-23 08:41:30,310 : INFO : collected 54008 word types and 7932 unique tags from a corpus of 7932 examples and 4866436 words
2020-02-23 08:41:30,311 : INFO : Loading a fresh vocabulary
2020-02-23 08:41:30,343 : INFO : effective_min_count=5 retains 15409 unique words (28% of original 54008, drops 38599)
2020-02-23 08:41:30,343 : INFO : effective_min_count=5 leaves 4811064 word corpus (98% of original 4866436, drops 55372)
2020-02-23 08:41:30,383 : INFO : deleting the raw counts dictionary of 54008 items
2020-02-23 08:41:30,385 : INFO : sample=0.001 downsamples 51 most-common words
2020-02-23 08:41:30,386 : INFO : downsampling leaves estimated 4105293 word corpus (85.3% of prior 4811064)
2020-02-23 08:41:30,422 : INFO : estimated required memory for 15409 words and 300 dimensions: 54204500 bytes
2020-02-23 08:4

training epoch 0 ...


2020-02-23 08:41:35,088 : INFO : EPOCH 1 - PROGRESS: at 37.10% examples, 1515933 words/s, in_qsize 10, out_qsize 0
2020-02-23 08:41:36,092 : INFO : EPOCH 1 - PROGRESS: at 75.08% examples, 1532878 words/s, in_qsize 9, out_qsize 0
2020-02-23 08:41:36,818 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-02-23 08:41:36,821 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-02-23 08:41:36,822 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-23 08:41:36,825 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-23 08:41:36,827 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-23 08:41:36,827 : INFO : EPOCH - 1 : training on 4866436 raw words (4113463 effective words) took 2.7s, 1495929 effective words/s
2020-02-23 08:41:36,828 : INFO : training on a 4866436 raw words (4113463 effective words) took 2.8s, 1493561 effective words/s
2020-02-23 08:41:36,830 : INFO : precomputing L2-n

found similar document with id 7771 in position 0 with similarity score 0.618470311164856
found similar document with id 4449 in position 0 with similarity score 0.6838314533233643
found similar document with id 7650 in position 0 with similarity score 0.6442806124687195
found similar document with id 4448 in position 0 with similarity score 0.7068660259246826
found similar document with id 960 in position 0 with similarity score 0.6356767416000366
found similar document with id 4337 in position 0 with similarity score 0.6365523338317871
found similar document with id 3311 in position 0 with similarity score 0.6094967126846313
found similar document with id 7759 in position 0 with similarity score 0.5914686322212219
mean percentiles: 80.00
========== Saving best model with mean_percentiles: 80.00 ==========


2020-02-23 08:41:37,407 : INFO : saved model/alodokter-articles-doc2vec.model
2020-02-23 08:41:37,408 : WARNING : Effective 'alpha' higher than previous training cycles
2020-02-23 08:41:37,408 : INFO : training model with 5 workers on 15409 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=2


...

found similar document with id 7512 in position 0 with similarity score 0.5821084976196289
found similar document with id 3810 in position 0 with similarity score 0.5245790481567383
found similar document with id 5674 in position 0 with similarity score 0.582007884979248
found similar document with id 1349 in position 7 with similarity score 0.4493425786495209
found similar document with id 1311 in position 4 with similarity score 0.4602513909339905
found similar document with id 3572 in position 0 with similarity score 0.4742148518562317
found similar document with id 2951 in position 0 with similarity score 0.4879973530769348
found similar document with id 6860 in position 0 with similarity score 0.4633292257785797
mean percentiles: 69.00
current mean_percentiles: 69.00, best: 100.00
training epoch 9 ...


2020-02-23 08:42:02,147 : INFO : EPOCH 1 - PROGRESS: at 36.54% examples, 1502694 words/s, in_qsize 10, out_qsize 0
2020-02-23 08:42:03,149 : INFO : EPOCH 1 - PROGRESS: at 73.84% examples, 1515827 words/s, in_qsize 10, out_qsize 1
2020-02-23 08:42:03,782 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-02-23 08:42:03,789 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-02-23 08:42:03,790 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-23 08:42:03,791 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-23 08:42:03,794 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-23 08:42:03,795 : INFO : EPOCH - 1 : training on 4866436 raw words (4113005 effective words) took 2.7s, 1551364 effective words/s
2020-02-23 08:42:03,795 : INFO : training on a 4866436 raw words (4113005 effective words) took 2.7s, 1549290 effective words/s
2020-02-23 08:42:03,817 : WARNING : Effective 'al

found similar document with id 402 in position 1 with similarity score 0.39155665040016174
found similar document with id 4305 in position 0 with similarity score 0.4275767207145691
found similar document with id 4799 in position 0 with similarity score 0.4248434901237488
found similar document with id 775 in position 4 with similarity score 0.3866235017776489
found similar document with id 2293 in position 9 with similarity score 0.3794844448566437
found similar document with id 6129 in position 0 with similarity score 0.5287708640098572
found similar document with id 694 in position 7 with similarity score 0.3622402250766754
found similar document with id 3377 in position 0 with similarity score 0.46711117029190063
found similar document with id 6086 in position 0 with similarity score 0.4845796227455139
mean percentiles: 69.00
current mean_percentiles: 69.00, best: 100.00
training epoch 10 ...


2020-02-23 08:42:04,827 : INFO : EPOCH 1 - PROGRESS: at 35.70% examples, 1461237 words/s, in_qsize 10, out_qsize 1
2020-02-23 08:42:05,829 : INFO : EPOCH 1 - PROGRESS: at 75.47% examples, 1546078 words/s, in_qsize 9, out_qsize 0
2020-02-23 08:42:06,444 : INFO : worker thread finished; awaiting finish of 4 more threads
2020-02-23 08:42:06,445 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-02-23 08:42:06,450 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-02-23 08:42:06,454 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-02-23 08:42:06,456 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-02-23 08:42:06,456 : INFO : EPOCH - 1 : training on 4866436 raw words (4113866 effective words) took 2.6s, 1561320 effective words/s
2020-02-23 08:42:06,456 : INFO : training on a 4866436 raw words (4113866 effective words) took 2.6s, 1559173 effective words/s


found similar document with id 557 in position 4 with similarity score 0.4253980815410614
found similar document with id 6400 in position 0 with similarity score 0.4955461621284485
found similar document with id 5756 in position 0 with similarity score 0.47075992822647095
found similar document with id 4463 in position 2 with similarity score 0.4597526490688324
found similar document with id 1244 in position 3 with similarity score 0.3155973255634308
found similar document with id 467 in position 3 with similarity score 0.44359251856803894
found similar document with id 1760 in position 0 with similarity score 0.5016111731529236
found similar document with id 4002 in position 3 with similarity score 0.44518810510635376
mean percentiles: 65.00
current mean_percentiles: 65.00, best: 100.00
early stop...
========== Saving best model with mean_percentiles: 100.00 ==========
